# Applying Avolio et al. 2019 to MVCO Abundance Data

We need methods to bridge multivariate measures (too abstract and tricky to interpret wrt to community change) with community structure and composition.

- species evenness: relative abundance of species in a community
- species richness: number of species in a community


## Rank Abundance Curves

### What's a rank abundance curve?

A rank abundance curve is plot of abundance rank (most abundant is 1st, second most is rank 2 and so forth) vs. species abundance.


The library codyn has functions that are designed to use RACs:
- RAC_change()
- RAC_diffference()

They extend the existing rank_change() and turnover() functions in codyn.

RAC_change() computes change in species evenness:

$$ \Delta S = \frac{S_{t+1}-S_{t}}{S_{total}}$$ where 

$S_{t} = $ richness at time t at replicate

$S_{t+1} = $ richness at time t + 1 at same replicate

$S_{total} =$ total number of unique species at both time periods


RAC_differrence computes species richness difference

$$ S.D. = \frac{S_x - S_y}{S_{total}}$$

$S_x$ and $S_y$ are the levels species richness at 2 replicates

$S_{total}$ is total number of unique species at both replicates.


NOTE:

$\Delta S$ and $S.D$ are proportions bound between -1 and 1.


community_structure() allows you specify 3 eveness measures.
1. inverse of Simpson's D
    - $P_i = \frac{x_i}{\sum x}$
    - $D = \sum_{x_i}^{S}P_i$
    - SimpEven = $\frac{\frac{1}{D}}{S}$ where $x_i$ is the abundance of species i
2. $E_Q$ a measure of the slope of an RAC and converts it to a 0-1 scale. if the slope is relatively flat, evenness is high and if it is steep evenness is low.

$$ E_Q = \frac{-2}{\pi*arctan(b')}$$ where $b'$ is the slope of the line fitted by a linear model of relationship between the relative rank of a species vs. its log abundance.

NOTE: This paper likes $E_Q$ bc it doesn't report an evenness of 1 if only 1 species is present like the first method and has a more normal distribution of values than $E_{var}$ .

3. $E_{var}$ a measure of the variance of abundance values normalized to 1.
    $$E_{var} = 1 - \frac{2}{\pi} a tan(\frac{s-1}{x}var(lnx))$$
    
    where s is the number of species in the sample and var is the sample variance. WHAT IS X tho???


In [1]:
# install.packages("codyn")

library("dplyr") #for wrangling dataframes
library("tidyr") #for wrangling dataframes
library("lubridate") # for working with dates
library("codyn") #for RAC functions
library("ggplot2")



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## $\Delta RAC$ on MVCO data

In [4]:
dfBv = read.csv('/D/MIT-WHOI/data/2021/biovol_concentration_by_class_time_series_CNN_daily.csv') #daily biovol concentration
dfConc = read.csv('/D/MIT-WHOI/data/2021/concentration_by_class_time_series_CNN_daily.csv') #daily biovol concentration

dfBv <- dfBv  %>% select(-c("bead","fiber","camera_spot","detritus","detritus_clear","fiber_TAG_external_detritus" ,"unknown2","square_unknown","mix","bubble","pollen","fecal_pellet"))
dfConc <- dfConc  %>% select(-c("bead","fiber","camera_spot","detritus","detritus_clear","fiber_TAG_external_detritus" ,"unknown2","square_unknown","mix","bubble","pollen","fecal_pellet"))
print(names(dfBv))

## converting to long format
# print(head(dfBv))
dfBv_long <- gather(dfBv,species,conc,Acanthoica_quattrospina:zooplankton,factor_key=TRUE)
dfConc_long <- gather(dfConc,species,conc,Acanthoica_quattrospina:zooplankton,factor_key=TRUE)


dfBv_long$date = dmy_hms(as.character(dfBv_long$datetime))
dfBv_long$year = year(dfBv_long$date)
dfBv_long$month = month(dfBv_long$date)
dfBv_long$day = day(dfBv_long$date)
dfBv_long["mdy"] = paste0(dfBv_long$month,"_",dfBv_long$day,"_",dfBv_long$year)

head(dfBv_long)

dfBv_long<- dfBv_long %>% 
            drop_na() %>%
            filter_all( all_vars(. != 0)) %>%
            group_by(mdy,species) %>%
            summarize(conc.sum = sum(conc))
head(dfBv_long)

dfConc_long$date = dmy_hms(as.character(dfConc_long$datetime))
dfConc_long$year = year(dfConc_long$date)
dfConc_long$month = month(dfConc_long$date)
dfConc_long$day = day(dfConc_long$date)
dfConc_long["mdy"] = paste0(dfConc_long$month,"_",dfConc_long$day,"_",dfConc_long$year)

dfConc_long<- dfConc_long %>% 
            drop_na() %>%
            filter_all( all_vars(. != 0)) %>%
            group_by(mdy,species) %>%
            summarize(conc.sum = sum(conc))


  [1] "datetime"                                    
  [2] "Acanthoica_quattrospina"                     
  [3] "Akashiwo"                                    
  [4] "Alexandrium_catenella"                       
  [5] "Amphidinium"                                 
  [6] "Amylax"                                      
  [7] "Apedinella"                                  
  [8] "Asterionellopsis_glacialis"                  
  [9] "Bacillaria"                                  
 [10] "Bacillariophyceae"                           
 [11] "Bacteriastrum"                               
 [12] "Balanion"                                    
 [13] "Biddulphia"                                  
 [14] "Calciopappus"                                
 [15] "Calciosolenia_brasiliensis"                  
 [16] "Cerataulina_pelagica"                        
 [17] "Ceratium"                                    
 [18] "Ceratium_furca"                              
 [19] "Ceratium_fusus"                        

,datetime,species,conc,date,year,month,day,mdy
,<fct>,<fct>,<dbl>,<dttm>,<dbl>,<dbl>,<int>,<chr>
1,06-Jun-2006 21:26:01,Acanthoica_quattrospina,9.759605,2006-06-06 21:26:01,2006,6,6,6_6_2006
2,07-Jun-2006 11:58:20,Acanthoica_quattrospina,290.813132,2006-06-07 11:58:20,2006,6,7,6_7_2006
3,08-Jun-2006 12:30:33,Acanthoica_quattrospina,192.535218,2006-06-08 12:30:33,2006,6,8,6_8_2006
4,09-Jun-2006 07:29:59,Acanthoica_quattrospina,147.161169,2006-06-09 07:29:59,2006,6,9,6_9_2006
5,10-Jun-2006 15:24:26,Acanthoica_quattrospina,187.400310,2006-06-10 15:24:26,2006,6,10,6_10_2006
6,11-Jun-2006 11:57:42,Acanthoica_quattrospina,166.991430,2006-06-11 11:57:42,2006,6,11,6_11_2006


`summarise()` has grouped output by 'mdy'. You can override using the `.groups` argument.



mdy,species,bvsum
<chr>,<fct>,<dbl>
1_1_2007,Acanthoica_quattrospina,156.05300
1_1_2007,Alexandrium_catenella,199.51739
1_1_2007,Amphidinium,148.82489
1_1_2007,Amylax,5.71656
1_1_2007,Apedinella,439.72254
1_1_2007,Asterionellopsis_glacialis,9100.79253


`summarise()` has grouped output by 'mdy'. You can override using the `.groups` argument.



In [7]:
#reading in long format of dataframe with just sorted functional groups
dfFunc_long= read.csv("/D/MIT-WHOI/github_repos/plankton-index/df_daily_long.txt",sep = " ")

dfFunc_long["datetime"] = dmy_hms(as.character(dfFunc_long$datetime))
dfFunc_long["day"] = as.numeric(day(dfFunc_long$datetime))
dfFunc_long$year = as.numeric(year(dfFunc_long$datetime))
dfFunc_long["month"] = as.numeric(month(dfFunc_long$datetime))
dfFunc_long["mdy"] = paste0(dfFunc_long$month,"_",dfFunc_long$day,"_",dfFunc_long$year)

dfFunc_long <- dfFunc_long %>% drop_na() %>% filter_all( all_vars(. != 0)) %>% group_by(mdy,group) %>% summarize(bvsum = sum(biovol.mL))
# print(dfFunc_long[dfFunc_long["month_day"]=="1_1_2007",])


`summarise()` has grouped output by 'month_day'. You can override using the `.groups` argument.



# A tibble: 11 × 3
# Groups:   month_day [1]
   month_day group               bvsum
   <chr>     <fct>               <dbl>
 1 1_1_2007  ""                   132.
 2 1_1_2007  "Ciliate"          83560.
 3 1_1_2007  "Coccolithophore"   1684.
 4 1_1_2007  "Diatom"          653771.
 5 1_1_2007  "Dinoflagellate"   36825.
 6 1_1_2007  "flagellate"       53125.
 7 1_1_2007  "IFCB artifact"     1461.
 8 1_1_2007  "Nano"              3208.
 9 1_1_2007  "Other live"        3587.
10 1_1_2007  "Other not alive"   2685.
11 1_1_2007  "pennatediatom"    38293.


In [7]:
RAC_change(df = dfFunc_long_grouped[1:100,], 
           species.var = "group",
           abundance.var = "bvsum",
           time.var ="month_day")


Warning message in RAC_change(df = dfFunc_long_grouped[1:100, ], species.var = "group", :
“evenness_change values contain NAs because there are plots with only one species”


month_day,month_day2,richness_change,evenness_change,rank_change,gains,losses
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1_1_2007,1_1_2008,0.0000000,0.013706833,0.03305785,0,0.0000000
1_1_2008,1_1_2009,0.0000000,-0.044177711,0.16528926,0,0.0000000
1_1_2009,1_1_2010,0.0000000,0.015966852,0.09917355,0,0.0000000
1_1_2010,1_1_2011,0.0000000,-0.002151787,0.06611570,0,0.0000000
1_1_2011,1_1_2012,0.0000000,0.034369745,0.08264463,0,0.0000000
1_1_2012,1_1_2013,0.0000000,0.007566854,0.08264463,0,0.0000000
1_1_2013,1_1_2014,0.0000000,0.071484239,0.11570248,0,0.0000000
1_1_2014,1_1_2015,0.0000000,-0.028026122,0.09917355,0,0.0000000
1_1_2015,1_1_2016,-0.9090909,NA,0.38842975,0,0.9090909


In [8]:
data(pplots)
# Without replicates
df <- subset(pplots, plot == 25)
RAC_change(df = df,
           species.var = "species",
           abundance.var = "relative_cover",
           time.var = "year")

# With replicates
df <- subset(pplots, year < 2004 & plot %in% c(6, 25, 32))
RAC_change(df = df,
           species.var = "species",
           abundance.var = "relative_cover",
           replicate.var = "plot",
           time.var = "year")
           
# With reference year
df <- subset(pplots, year < 2005 & plot %in% c(6, 25, 32))
RAC_change(df = df,
           species.var = "species",
           abundance.var = "relative_cover",
           replicate.var = "plot",
           time.var = "year",
           reference.time = 2002)

# }

year,year2,richness_change,evenness_change,rank_change,gains,losses
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2002,2003,-0.1500000,-0.05537606,0.145000,0.10000000,0.25000000
2003,2004,0.2380952,0.04096495,0.122449,0.28571429,0.04761905
2004,2005,-0.1904762,-0.05519765,0.106576,0.04761905,0.23809524


year,year2,plot,richness_change,evenness_change,rank_change,gains,losses
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2002,2003,6,-0.20,-0.07990473,0.120,0.08,0.28
2002,2003,25,-0.15,-0.05537606,0.145,0.10,0.25
2002,2003,32,0.04,0.05890419,0.128,0.20,0.16


year,year2,plot,richness_change,evenness_change,rank_change,gains,losses
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2002,2003,6,-0.20000000,-0.07990473,0.1200000,0.0800000,0.2800000
2002,2003,25,-0.15000000,-0.05537606,0.1450000,0.1000000,0.2500000
2002,2003,32,0.04000000,0.05890419,0.1280000,0.2000000,0.1600000
2002,2004,6,-0.03571429,0.02686011,0.1683673,0.1785714,0.2142857
2002,2004,25,0.08333333,-0.01424465,0.1302083,0.2500000,0.1666667
2002,2004,32,-0.08333333,0.06876699,0.1493056,0.1666667,0.2500000


## Comparing RAC curves


1. relative rank
2. cummulative community abundance
3. then a plot is made of relative rank vs. cummulative abundance is plotted with stepwise fuction ($f_{k,t}(r)$ for relative rank r of the community at location k and the time t.

$$\Delta Curve = \sum_{i =1}^{N}| f_{k,t}(r_i)-f_{k,t+1}(r_i)|(r_i - r_{i-1})$$

where N = number of unique relative ranks
r = sor

In [30]:
??curve_change()

In [ ]:
### Mutivariate composition comparisions

1. Distance between centroids
2. dispersion around centroids

In [ ]:
multivariate_change()